Describe invertible calculus.
first examples $(f(x), [b_{1}, b_{1}, ...])$ give concrete examples
then given $f(x) \in \mathcal{O}(\mathbb{C})$ then $f(x)$ is defined by a taylor series $[a_0, a_1, a_2, ...]$
so $(f(x), [b_{1}, b_{2}, ...]) = ([a_1, a_2, a_3, ...], [b_1, b_2, ...]) = ([..., a_{-2}, a_{-1}, a_0, a_1, a_2, ...])$ where $b_n = a_{-n}$

In [ ]:
import Graphics.Rendering.Chart.Easy
import Graphics.Rendering.Chart.Backend.Cairo

In [ ]:
type Function = (Double -> Double)

_AnalyticAccuracy = 50

class Differentiable a where
    derivative :: a -> a
    antiderivative :: a -> a

class Analytic a where
    toFunction :: a -> Function

repeat' = concat . repeat

linspace :: Double -> Double -> Int -> [Double]
linspace start stop num =
    let dx = (stop - start)/fromIntegral (num - 1)
    in [fromIntegral i*dx + start| i <- [0 .. (num - 1)]]

factorial :: Integer -> Integer
factorial 0 = 1
factorial n = n*factorial (n - 1)

In [ ]:
data TaylorSeries = Constant Double | TaylorSeries [Double]

instance Show TaylorSeries where
    show (TaylorSeries terms) = "Taylor Series: " ++ init (show (take 10 terms)) ++ " ... ]"
    show (Constant a) = "Constant: " ++ show a

instance Num TaylorSeries where
    (+) (Constant a) (Constant b) = Constant (a + b)
    (+) (Constant a) (TaylorSeries (b:terms)) = TaylorSeries (a+b:terms)
    (+) function (Constant a) = Constant a + function 
    (+) (TaylorSeries termsA) (TaylorSeries termsB) = TaylorSeries [a+b | (a, b) <- zip termsA termsB]
    
    negate (Constant a) = Constant $ negate a
    negate (TaylorSeries terms) = TaylorSeries $ map negate terms
    
    abs (Constant a) = Constant $ abs a
    abs (TaylorSeries terms) = TaylorSeries $ map abs terms
    
    fromInteger a = Constant $ fromInteger a
    
    (*) (Constant a) (Constant b) = Constant (a*b)
    (*) (Constant a) (TaylorSeries terms) = TaylorSeries $ map (a*) terms
    (*) function (Constant a) = Constant a * function 
    (*) _ _ = error "Undefined"
    signum _ = error "Undefined"

instance Analytic TaylorSeries where
    toFunction (TaylorSeries taylorSeries) =
        let function :: TaylorSeries -> Function
            function (TaylorSeries taylorSeries) x = sum [a*x^^n/fromIntegral (factorial n) | (a, n) <- zip taylorSeries [0..] ]
        in function $ TaylorSeries (take _AnalyticAccuracy taylorSeries)

instance Differentiable TaylorSeries where
    derivative (TaylorSeries (a:function)) = TaylorSeries function

    antiderivative (TaylorSeries function) = TaylorSeries (0:function)

In [ ]:
data ShiftFunction = Scalar Double | ShiftFunction [Double] [Double]

instance Show ShiftFunction where
    show (ShiftFunction (a_0:terms) intterms) = "Shift function: [... " ++ (init.tail.show.reverse.take 4) intterms ++ ",(" ++ show a_0 ++ ")," ++ (init.tail.show.take 4) terms ++ " ...]"
    show (Scalar a) = "Constant: " ++ show a

instance Num ShiftFunction where
    (+) (Scalar a) (Scalar b) = Scalar (a + b)
    (+) (Scalar a) (ShiftFunction (b:terms) intterms) = ShiftFunction (a+b:terms) intterms
    (+) function (Scalar a) = Scalar a + function 
    (+) (ShiftFunction termsA inttermsA) (ShiftFunction termsB inttermsB) = ShiftFunction [a+b | (a, b) <- zip termsA termsB] [a+b | (a, b) <- zip inttermsA inttermsB]
    
    negate (Scalar a) = Scalar $ negate a
    negate (ShiftFunction terms intterms) = ShiftFunction (map negate terms) (map negate intterms)
    
    abs (Scalar a) = Scalar $ abs a
    abs (ShiftFunction terms intterms) = ShiftFunction (map abs terms) (map abs intterms)
    
    fromInteger a = Scalar $ fromInteger a
    
    (*) (Scalar a) (Scalar b) = Scalar (a*b)
    (*) (Scalar a) (ShiftFunction terms intterms) = ShiftFunction (map (a*) terms) (map (a*) intterms)
    (*) function (Scalar a) = Scalar a * function 
    (*) _ _ = error "Undefined"
    signum _ = error "Undefined"

instance Analytic ShiftFunction where
    toFunction (ShiftFunction terms intterms) =
        let function :: ShiftFunction -> Function
            function (ShiftFunction terms _) x = sum [a*x^^n/fromIntegral (factorial n) | (a, n) <- zip terms [0..] ]
        in function $ ShiftFunction (take _AnalyticAccuracy terms) []

instance Differentiable ShiftFunction where
    derivative (ShiftFunction (a:terms) intterms) = ShiftFunction terms (a:intterms)

    antiderivative (ShiftFunction terms (a:intterms)) = ShiftFunction (a:terms) intterms

In [ ]:
data StackFunction = StackFunction ShiftFunction String

instance Show StackFunction where
    show (StackFunction (Scalar a) _) = show (Scalar a)
    show (StackFunction (ShiftFunction _ intterms) function) = "StackFunction: (" ++ function ++ ",[" ++ (init.tail.show.take 10) intterms ++ "...])"

instance Num StackFunction where
    (+) (StackFunction functionA labelA) (StackFunction functionB labelB) = StackFunction (functionA + functionB) (labelA ++ "+" ++ labelB)
    
    negate (StackFunction function label) = StackFunction (-function) ("-" ++ label)
    
    abs (StackFunction function label) = StackFunction (abs function) ("abs(" ++ label ++ ")")
    
    fromInteger a = StackFunction (fromInteger a) (show a)
    
    (*) (StackFunction functionA labelA) (StackFunction functionB labelB) = StackFunction (functionA*functionB) (labelA ++ "*" ++ labelB)
    signum _ = error "Undefined"

instance Analytic StackFunction where
    toFunction (StackFunction function _) = toFunction function

instance Differentiable StackFunction where
    derivative (StackFunction function label) = StackFunction (derivative function) ("(d/dx)" ++ label)

    antiderivative (StackFunction function label) = StackFunction (antiderivative function) ("(d/dx)^(-1)" ++ label)

In [ ]:
eval :: (Analytic a) => a -> [Double] -> [(Double, Double)]
eval function xs = [(x, toFunction function x) | x<-xs]
plot' x function = toRenderable (plot (line "func" [eval function x]))
plots x functions = toRenderable (plot (line "func" [eval function x | function <- functions]))

exp' = TaylorSeries $ repeat 1
sin' = TaylorSeries $ repeat' [0, 1, 0, -1]
cos' = TaylorSeries $ repeat' [1, 0, -1, 0]

exp'' = ShiftFunction (repeat 1) (repeat 1)
sin'' = ShiftFunction (repeat' [0, 1, 0, -1]) (repeat' [-1, 0, 1, 0])
cos'' = ShiftFunction (repeat' [1, 0, -1, 0]) (repeat' [0, -1, 0, 1])

exp''' = StackFunction exp'' "exp(x)"
cos''' = StackFunction cos'' "cos(x)"
sin''' = StackFunction sin'' "sin(x)"

In [ ]:
x = linspace (-10) 4 500

func' = 6*cos' + 2*exp' - 2*Constant pi
func'' = 6*cos'' + 2*exp'' - 2*Scalar pi
func''' = 6*cos''' + 2*exp''' - 2*StackFunction (Scalar pi) "pi"
func = func'''
print func
plot' x func
plot' x $ derivative func
plot' x $ antiderivative func

In [ ]:
f = ShiftFunction (repeat 1) (repeat 1)
(derivative.derivative) f - 2*derivative f + 3*f

differential function $f'' - 2*f' + 3*f = 0$, try $f = e^{ax}$, then $a^2f - 2*a*f + 3*f = 0$ quadradic equation for ansatz $a^2 - 2*a + 3 = 0$ solution to ansatz $a = \frac{2+\sqrt{4-4*3}}{2}$ so $f(x) = 2e^x cos(x\sqrt{2})$

Recursive equation $a_{n+2} - 2*a_{n+1} + 3*a_n = 0$ rearanging then $a_n = \frac{2*a_{n+1} - a_{n+2}}{3}$, and $a_{n+2} = 2*a_{n+1} - 3*a_n$

In [ ]:
--terms :: (Num a) => a -> a -> Int -> a
--terms a b 0 = a
--terms a b 1 = b
--terms a b n = 2*terms a b (n - 1) - 3*terms a b (n-2)

--intterms :: (Fractional a) => a -> a -> Int -> a
--intterms a b 0 = a
--intterms a b 1 = b
--intterms a b n = (2*intterms a b (n + 1) - intterms a b (n + 2))/3

terms :: (Num a) => a -> a -> [a]
terms a b = let list = a:b:zipWith (\x y->2*y-3*x) list (tail list)
            in list

intterms :: (Fractional a) => a -> a -> [a]
intterms a b = let list = b:a:zipWith (\y x->(2*y - x)/3.0) (tail list) list
                in drop 2 list

(a, b) = (0, -10)

g = TaylorSeries $ terms a b
g' = ShiftFunction (terms a b) (intterms a b)

In [ ]:
x = linspace (-3) 3 100

print g'
print $ derivative g'
print $ antiderivative g'

plot' x g'
plot' x $ derivative g'
plot' x $ antiderivative g'

In [ ]:
gs = [TaylorSeries $ terms (-1) a | a <- linspace (-2) 2 10]
plots x gs
gs = [TaylorSeries $ terms a 0 | a <- linspace (-2) 0 10]
plots x gs
gs = [TaylorSeries $ terms a b | a <- linspace (-2) 2 10, b <- linspace (-2) 0 10]
plots x gs

In [ ]:
data Derivative = D | Dn Derivative deriving Eq
data Dx a = C a | Dx a Derivative deriving Eq

count :: Derivative -> Integer
count D = 1
count (Dn d) = 1 + count d

instance Show Derivative where
    show D = "dx"
    show d = "dx^" ++ show (count d)

instance (Show a, Num a, Eq a) => Show (Dx a) where
    show (C a) = show a
    show (Dx a d)
        | a == 1 = show d
        | a == 0 = show a
        | otherwise = show a ++ show d

instance Semigroup Derivative where
    (<>) D D = Dn D
    (<>) D (Dn dn) = Dn (Dn dn)
    (<>) d D = D <> d
    (<>) (Dn dn) d = dn <> Dn d

instance (Num a) => Semigroup (Dx a) where
    (<>) (C a) (C b) = C (a*b)
    (<>) (C a) (Dx b d) = Dx (a*b) d
    (<>) d (C a) = C a <> d
    (<>) (Dx a dn) (Dx b dm) = Dx (a*b) (dn<>dm)

instance (Num a) => Monoid (Dx a) where
    mempty = C 1

d = Dx 1.2 D
mappend d d

End of tests